# Ingest gene expression data from TCGA, TARGET and GTEX

Download gene expression data from the [UCSC Xena Toil re-compute dataset](https://xenabrowser.net/datapages/?host=https://toil.xenahubs.net), wrangle, enrich and store in an hdf5 file for machine learning. This dataset comprises gene expression data for nineteen thousand samples  processed using the same genomics pipeline and therefore comparable to each other. 

Each sample consists of a float32 vector, log2(tpm+0.001) normalized, of gene expression for each of 60k genes. Associated with these data is clinical information on each sample such as type (tumor vs. normal), primary site (where the sample came from in the human body) etc... We use this information to label the samples normal/0 vs. tumor/1 as well as to provide additional information for visualization and interpretation of models. 

The resulting hdf5 file, tumornormal.h5, contains the following datasets:

* **X_train, X_test:** float32 gene expression train/test sets split via stratification on class (primary site)
* **y_train, y_test:** Binary 0=Normal, 1=Tumor label for binary classification machine learning
* **features:** Ensemble gene id's for each feature in X
* **labels:** Labels for y binary values ie "Normal" = 0 and "Tumor" = 1
* **classes_train, classes_test:** Class id (integer) for each sample - useful for visualization when clustering
* **class_labels:** Text corresponding to each class id (ie disease) - useful as legend when visualizing
* **genes:** Hugo gene names corresponding to each feature (may be duplicates) - more recognizable when visualizing inner layers

In [17]:
import os
import requests
import numpy as np
import pandas as pd
import tables  # Required by h5py to write a pandas table
import h5py

In [18]:
%%time
"""
Download expression data from xena and save in an hdf5 file. This can take around 
30 minutes between the download and the conversion from tsv into float32 dataframes
We download manually vs. passing read_csv a url directly as the latter times
out with this size file. Note we invert the expression matrix to conform 
to machine learning where rows are samples vs. gene expression files where 
rows are features (gene, transcript, or variant) and columns are 
instances (sample or single cell)
"""
if not os.path.exists("data"):
    os.makedirs("data")
    
if not os.path.exists("data/TcgaTargetGtex_rsem_gene_tpm.tsv.gz"):
    print("Downloading TCGA, TARGET and GTEX expression data from UCSC Xena")
    r = requests.get("https://toil.xenahubs.net/download/TcgaTargetGtex_rsem_gene_tpm.gz")
    r.raise_for_status()
    with open("data/TcgaTargetGtex_rsem_gene_tpm.tsv.gz", "wb") as f:
        for chunk in r.iter_content(32768):
            f.write(chunk)

if not os.path.exists("data/TcgaTargetGtex_rsem_gene_tpm.hd5"):
    print("Converting expression to dataframe and storing in hdf5 file")
    expression = pd.read_csv("data/TcgaTargetGtex_rsem_gene_tpm.tsv.gz", 
                             sep="\t", index_col=0).dropna().astype(np.float32).T
    expression.to_hdf("data/TcgaTargetGtex_rsem_gene_tpm.hd5", "expression", mode="w", format="fixed")

X = pd.read_hdf("data/TcgaTargetGtex_rsem_gene_tpm.hd5", "expression").sort_index(axis=0)
print("X: samples={} genes={}".format(*X.shape))

X: samples=19260 genes=60498
CPU times: user 440 ms, sys: 42.5 s, total: 42.9 s
Wall time: 42.8 s


In [19]:
X.head()

sample,ENSG00000242268.2,ENSG00000259041.1,ENSG00000270112.3,ENSG00000167578.16,ENSG00000278814.1,ENSG00000078237.5,ENSG00000269416.5,ENSG00000263642.1,ENSG00000146083.11,ENSG00000158486.13,...,ENSG00000009694.13,ENSG00000238244.3,ENSG00000216352.1,ENSG00000123685.8,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.18,ENSG00000231119.2,ENSG00000280861.1,ENSG00000181518.3
GTEX-1117F-0226-SM-5GZZ7,-9.9658,-9.9658,-4.2934,5.1190,-9.9658,0.8488,-9.9658,-9.9658,5.1498,-9.9658,...,2.4571,-1.4305,-9.9658,2.8178,-0.1031,-9.9658,5.1631,-3.3076,-9.9658,-9.9658
GTEX-1117F-0426-SM-5EGHI,-9.9658,-9.9658,0.0014,4.1277,-9.9658,0.6880,-9.9658,-9.9658,3.4830,-9.9658,...,-0.9132,-9.9658,-9.9658,-0.9406,-9.9658,-1.5105,4.1764,-5.0116,-9.9658,-9.9658
GTEX-1117F-0526-SM-5EGHJ,-9.9658,-9.9658,-9.9658,4.4067,-9.9658,0.0440,-9.9658,-9.9658,4.3841,-9.9658,...,1.5165,-9.9658,-9.9658,1.7141,-1.1488,-9.9658,4.8768,-9.9658,-9.9658,-9.9658
GTEX-1117F-0626-SM-5N9CS,-1.2481,-9.9658,-5.5735,5.6860,-9.9658,1.3679,-9.9658,-9.9658,5.0644,-1.8836,...,1.5998,-9.9658,-9.9658,3.9356,-1.1488,-1.0559,4.8694,-1.9379,-9.9658,-9.9658
GTEX-1117F-0726-SM-5GIEN,-3.8160,-9.9658,0.3573,4.0357,-9.9658,-0.4325,-5.5735,-9.9658,3.9421,-3.4580,...,-3.1714,-9.9658,-9.9658,0.6608,-1.1811,-9.9658,3.6816,-2.6349,-9.9658,-9.9658


In [20]:
# Read in the sample labels from Xena ie clinical/phenotype information on each sample
Y = pd.read_table("https://toil.xenahubs.net/download/TcgaTargetGTEX_phenotype.txt.gz", compression="gzip",
                  header=0, names=["id", "category", "disease", "primary_site", "sample_type", "gender", "study"],
                  sep="\t", encoding="ISO-8859-1", index_col=0, dtype="str").sort_index(axis=0)

# Compute and add a tumor/normal column - TCGA and TARGET have some normal samples, GTEX is all normal.
Y["tumor_normal"] = Y.apply(
    lambda row: "Normal" if row["sample_type"] in ["Cell Line", "Normal Tissue", "Solid Tissue Normal"]
    else "Tumor", axis=1)

Y[0:100:4000].head()

,category,disease,primary_site,sample_type,gender,study,tumor_normal
id,,,,,,,
GTEX-1117F-0226-SM-5GZZ7,Adipose - Subcutaneous,Adipose - Subcutaneous,Adipose Tissue,Normal Tissue,Female,GTEX,Normal


In [21]:
Y.describe()

,category,disease,primary_site,sample_type,gender,study,tumor_normal
count,19130,19130,19126,19131,18972,19131,19131
unique,93,93,46,17,2,3,2
top,Breast Invasive Carcinoma,Breast Invasive Carcinoma,Brain,Primary Tumor,Male,TCGA,Tumor
freq,1212,1212,1846,9185,10456,10535,10531


In [22]:
# Use the tissue location as the class label for the purposes of stratification
class_attribute = "primary_site"

# Tumor vs. Normal is the binary attribute we'll use to train on
label_attribute = "tumor_normal"

In [23]:
# Remove rows where the class is null or the sample is missing
Y_not_null = Y[pd.notnull(Y[class_attribute])]
intersection = X.index.intersection(Y_not_null.index)
X_clean = X[X.index.isin(intersection)]
Y_clean = Y[Y.index.isin(intersection)]

# Make sure the label and example samples are in the same order
assert(X_clean.index.equals(Y_clean.index))

print(intersection.shape[0], "samples with non-null labels")

19126 samples with non-null labels


In [24]:
# Convert tumor/normal labels to binary 1/0
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_binary = encoder.fit_transform(Y_clean["tumor_normal"])

In [25]:
# Convert classes into numbers
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(Y_clean[class_attribute].values)
classes = encoder.transform(Y_clean[class_attribute])
print("Total classes for stratification:", len(set(classes)))
class_labels = encoder.classes_

Total classes for stratification: 46


In [26]:
%%time
# Split into stratified training and test sets based on classes (i.e. tissue type) so that we have equal
# proportions of each tissue type in the train and test sets
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(X_clean.values, Y_clean[class_attribute]):
    X_train, X_test = X.values[train_index], X_clean.values[test_index]
    y_train, y_test = y_binary[train_index], y_binary[test_index]
    classes_train, classes_test = classes[train_index], classes[test_index]

CPU times: user 1.96 s, sys: 1.4 s, total: 3.36 s
Wall time: 3.36 s


In [27]:
"""
Feature labels are ensemble ids, convert to hugo gene names for use in interpreting
hidden layers in any trained models as they are better known to most bioinformaticians 
and clinicians. We're using an assembled table from John Vivian @ UCSC here. Another
option would be ftp://ftp.ebi.ac.uk/pub/databases/genenames/new/tsv/hgnc_complete_set.txt
"""
ensemble_to_hugo = pd.read_table(
    "https://github.com/jvivian/docker_tools/blob/master/gencode_hugo_mapping/attrs.tsv?raw=true",
    index_col=0)
ensemble_to_hugo.head()

,geneName,geneType,geneStatus,transcriptId,transcriptName,transcriptType,transcriptStatus,havanaGeneId,havanaTranscriptId,ccdsId,level,transcriptClass
geneId,,,,,,,,,,,,
ENSG00000223972.5,DDX11L1,transcribed_unprocessed_pseudogene,KNOWN,ENST00000456328.2,DDX11L1-002,processed_transcript,KNOWN,OTTHUMG00000000961.2,OTTHUMT00000362751.1,NaN,2,pseudo
ENSG00000223972.5,DDX11L1,transcribed_unprocessed_pseudogene,KNOWN,ENST00000450305.2,DDX11L1-001,transcribed_unprocessed_pseudogene,KNOWN,OTTHUMG00000000961.2,OTTHUMT00000002844.2,NaN,2,pseudo
ENSG00000227232.5,WASH7P,unprocessed_pseudogene,KNOWN,ENST00000488147.1,WASH7P-001,unprocessed_pseudogene,KNOWN,OTTHUMG00000000958.1,OTTHUMT00000002839.1,NaN,2,pseudo
ENSG00000278267.1,MIR6859-1,miRNA,KNOWN,ENST00000619216.1,MIR6859-1-201,miRNA,KNOWN,NaN,NaN,NaN,3,nonCoding
ENSG00000243485.3,RP11-34P13.3,lincRNA,KNOWN,ENST00000473358.1,RP11-34P13.3-001,lincRNA,KNOWN,OTTHUMG00000000959.2,OTTHUMT00000002840.1,NaN,2,nonCoding


In [28]:
# The ensembl to hugo table has duplicates due to the many transcripts that map
# to a gene. Remove the duplicates and then lookup hugo for each ensemble id
hugo = ensemble_to_hugo[~ensemble_to_hugo.index.duplicated(keep='first')].reindex(X_clean.columns.values)["geneName"].fillna("")

# Make sure we end up with the order of features being identical as some ensemble id's
# map to the same hugo gene id
assert(X_clean.columns.equals(hugo.index))

In [30]:
"""
Write to an h5 file for training (see above for details on each dataset)
"""
with h5py.File("data/tumor_normal.h5", "w") as f:
    f.create_dataset('X_train', X_train.shape, dtype='f')[:] = X_train
    f.create_dataset('X_test', X_test.shape, dtype='f')[:] = X_test
    f.create_dataset('y_train', y_train.shape, dtype='i')[:] = y_train
    f.create_dataset('y_test', y_test.shape, dtype='i')[:] = y_test
    f.create_dataset('classes_train', y_train.shape, dtype='i')[:] = classes_train
    f.create_dataset('classes_test', y_test.shape, dtype='i')[:] = classes_test
    f.create_dataset('features', X_clean.columns.shape, 'S10', 
                     [l.encode("ascii", "ignore") for l in X_clean.columns.values])
    f.create_dataset('genes', hugo.shape, 'S10', 
                     [l.encode("ascii", "ignore") for l in hugo.values.tolist()])
    f.create_dataset('labels', (2, 1), 'S10', 
                     [l.encode("ascii", "ignore") for l in ["Normal", "Tumor"]])
    f.create_dataset('class_labels', (len(class_labels), 1), 'S10', 
                     [l.encode("ascii", "ignore") for l in class_labels])